### If you wish to visualize the results from the code below, you will need to have the `geemap` package installed

In [1]:
import ee
import geemap
from RadGEEToolbox import LandsatCollection
from RadGEEToolbox import GetPalette
from RadGEEToolbox import VisParams

In [2]:
# Attempt to initialize Earth Engine
try:
    ee.Initialize()
    print("Earth Engine initialized successfully.")
except Exception as e:
    print("Initialization failed, attempting authentication...")
    try:
        ee.Authenticate()
        ee.Initialize()
        print("Authentication and initialization successful.")
    except Exception as auth_error:
        print("Authentication failed. Error details:", auth_error)

Earth Engine initialized successfully.


### RadGEEToolbox hosts two packages to help image visualization: 1) GetPalette and 2) VisParams
- GetPalette has the get_palette() function which allows for easily getting the hex series for a variety of palettes
- VisParams has the get_visualization_params() function which provides an alternative to visualization parameter dictionaries

In [3]:
# Example getting the jet palette
# options: 'algae', 'dense', 'greens', 'haline', 'inferno', 'jet', 'matter', 'pubu', 'soft_blue_green_red', 'thermal', 'turbid', 'ylord'

jet = GetPalette.get_palette('jet')
print(jet)

['#00007F', '#002AFF', '#00D4FF', '#7FFF7F', '#FFD400', '#FF2A00', '#7F0000']


#### get_visualization_params() takes satellite, index, min_val, max_val, and palette as arguments
- min_val, max_val, and palette are optional, however, as defaults are provided based on the satellite and index
- providing optional values will override the defaults

Options for satellite: 'Landsat', 'landsat', 'Sentinel2', or 'sentinel2'

Options for index: 'TrueColor', 'NDVI', 'NDWI', 'halite', 'gypsum', 'LST', 'NDTI', 'KIVU', or '2BDA'

In [4]:
true_color_vis_params = VisParams.get_visualization_params(satellite='landsat', index='TrueColor')
print(true_color_vis_params)
ndvi_vis_params = VisParams.get_visualization_params(satellite='landsat', index='NDVI')
print(ndvi_vis_params)

{'min': 0, 'max': 30000, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}
{'min': 0, 'max': 0.5, 'bands': ['ndvi'], 'palette': ['#f7fcf5', '#e5f5e0', '#c7e9c0', '#a1d99b', '#74c476', '#41ab5d', '#238b45', '#006d2c', '#00441b']}


### Pulling it all together
#### Lets 1) Define a collection, 2) Process the collection for various products (ndwi, median, LST), 3) Get list of dates of images, 4) Visualize most recent image as true color and singeband products

In [5]:
col = LandsatCollection('2023-06-01', '2023-07-30', [32, 33], 38, 20)
col = col.MosaicByDate
ndwi = col.ndwi
LST = col.LST
mean_temperature = LST.mean
dates = col.dates
print(dates)

['2023-06-21', '2023-06-29', '2023-07-07', '2023-07-15']


True Color Visualization

In [6]:
Map = geemap.Map(center=(40.7514, -111.9064), zoom=9)
Map.addLayer(col.image_grab(-1), vis_params=VisParams.get_visualization_params('Landsat', 'TrueColor'))
Map

Map(center=[40.7514, -111.9064], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

NDWI Visualization

In [7]:
Map = geemap.Map(center=(40.7514, -111.9064), zoom=9)
# min and max values are optional but allow for customization
Map.addLayer(ndwi.image_grab(-1), vis_params=VisParams.get_visualization_params('Landsat', 'NDWI', min_val=-0.1, max_val=0.1))
Map.add_colorbar(vis_params=VisParams.get_visualization_params('Landsat', 'NDWI', min_val=-0.1, max_val=0.1))
Map

Map(center=[40.7514, -111.9064], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

Land Surface Temperature Visualization

In [8]:
Map = geemap.Map(center=(40.7514, -111.9064), zoom=9)
vis_params=VisParams.get_visualization_params('Landsat', 'LST', min_val=5, max_val=60)
Map.addLayer(LST.image_grab(-1), vis_params=vis_params)
Map.add_colorbar(vis_params=vis_params)
Map

Map(center=[40.7514, -111.9064], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

Mean LST image visualization

In [9]:
Map = geemap.Map(center=(40.7514, -111.9064), zoom=9)
vis_params=VisParams.get_visualization_params('Landsat', 'LST', min_val=5, max_val=60)
Map.addLayer(mean_temperature, vis_params=vis_params)
Map.add_colorbar(vis_params=vis_params)
Map

Map(center=[40.7514, -111.9064], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

#### Finally, lets look at a chained example looking at turbidity in water pixels - but let's do it taking full advantage of the package
##### Look how little code is needed to define the collection using multiple landsat tiles, mosaic images with the same date, mask the collection to water pixels, and calculate the relative turbidity of the water pixels - then display it!

Note the location and date are for demonstrative purposes only.

In [10]:
masked_turbidity_col = LandsatCollection('2023-04-01', '2023-07-30', [32, 33], 38, 20).MosaicByDate.masked_to_water_collection.turbidity
image = masked_turbidity_col.image_grab(-1)
print(image.getInfo())

{'type': 'Image', 'bands': [{'id': 'ndti', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '8', 'Date_Filter': '2023-07-15', 'SPACECRAFT_ID': 'LANDSAT_9', 'CLOUD_COVER': 0.045, 'SENSOR_ID': 'OLI_TIRS', 'PROCESSING_LEVEL': 'L2SP'}}


In [11]:
Map = geemap.Map(center=(40.7514, -111.9064), zoom=9)
vis_params=VisParams.get_visualization_params('Landsat', 'NDTI')
Map.addLayer(image, vis_params=vis_params)
Map.add_colorbar(vis_params=vis_params)
Map

Map(center=[40.7514, -111.9064], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

### *Please refer to the [RadGEEToolbox documentation](https://radgeetoolbox.readthedocs.io/en/latest/) for more information and a comprehensive list of available functionality*